In [7]:
from google.colab import files
uploaded = files.upload()


Saving tab_aml_polars_version_fourtneen.py to tab_aml_polars_version_fourtneen.py


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
DATASET_DIR = "/content/drive/MyDrive/kaggle_datasets/SAML-D"

In [3]:
import polars as pl


In [4]:
df = pl.read_csv(f"{DATASET_DIR}/SAML-D.csv")
df.head()

Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,Laundering_type
str,str,i64,i64,f64,str,str,str,str,str,i64,str
"""10:35:19""","""2022-10-07""",8724731955,2769355426,1459.15,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,"""Normal_Cash_Deposits"""
"""10:35:20""","""2022-10-07""",1491989064,8401255335,6019.64,"""UK pounds""","""Dirham""","""UK""","""UAE""","""Cross-border""",0,"""Normal_Fan_Out"""
"""10:35:20""","""2022-10-07""",287305149,4404767002,14328.44,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cheque""",0,"""Normal_Small_Fan_Out"""
"""10:35:21""","""2022-10-07""",5376652437,9600420220,11895.0,"""UK pounds""","""UK pounds""","""UK""","""UK""","""ACH""",0,"""Normal_Fan_In"""
"""10:35:21""","""2022-10-07""",9614186178,3803336972,115.25,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,"""Normal_Cash_Deposits"""


In [8]:
import torch
from tab_aml_polars_version_fourtneen import preprocess_saml_d, fast_hash, recast, collate_encoded_batch, custom_split_polars, TabAMLDataset, TabAMLModel, build_weighted_sampler, compute_pos_weight, train_epoch, evaluate
from torch.utils.data import DataLoader
import torch.nn as nn

In [9]:
df = preprocess_saml_d(df)

In [10]:
df = recast(df)

In [11]:
df.head(5)

Date,Sender_account,Receiver_account,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,timestamp,day,month,year,hour,day_of_week,is_weekend,amount_log
date,i64,i64,str,str,str,str,str,i8,datetime[μs],i8,i8,i16,i8,i8,i8,f64
2022-10-07,8724731955,2769355426,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022-10-07 10:35:19,7,10,2022,10,5,1,7.286294
2022-10-07,1491989064,8401255335,"""UK pounds""","""Dirham""","""UK""","""UAE""","""Cross-border""",0,2022-10-07 10:35:20,7,10,2022,10,5,1,8.702949
2022-10-07,287305149,4404767002,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cheque""",0,2022-10-07 10:35:20,7,10,2022,10,5,1,9.570071
2022-10-07,5376652437,9600420220,"""UK pounds""","""UK pounds""","""UK""","""UK""","""ACH""",0,2022-10-07 10:35:21,7,10,2022,10,5,1,9.383957
2022-10-07,9614186178,3803336972,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022-10-07 10:35:21,7,10,2022,10,5,1,4.755743


In [12]:
train_df, val_df, test_df = custom_split_polars(df, validation_dt=70, test_dt=35)


In [13]:
print(len(train_df), len(val_df), len(test_df))

7429619 1044845 1030388


In [14]:
cat_cols = ["Sender_account", "Receiver_account", "Payment_currency", "Received_currency", "Sender_bank_location",
            "Receiver_bank_location", "Payment_type", "day", "month", "year", "hour", "day_of_week", "is_weekend"]

In [15]:
cont_cols = ["amount_log"]

In [16]:
train_ds = TabAMLDataset(train_df, cat_cols, cont_cols, "Is_laundering", fit=True)

⚡ Using hashing for high-cardinality column: Sender_account
⚡ Using hashing for high-cardinality column: Receiver_account
[DEBUG] x_cat shape: (7429619, 13)


In [17]:
val_ds   = TabAMLDataset(val_df, cat_cols, cont_cols, "Is_laundering", encoders=train_ds.encoders, scaler=train_ds.scaler, fit=False)

⚡ Using hashing for high-cardinality column: Sender_account
⚡ Using hashing for high-cardinality column: Receiver_account
[DEBUG] x_cat shape: (1044845, 13)


In [18]:
test_ds  = TabAMLDataset(test_df, cat_cols, cont_cols, "Is_laundering", encoders=train_ds.encoders, scaler=train_ds.scaler, fit=False)


⚡ Using hashing for high-cardinality column: Sender_account
⚡ Using hashing for high-cardinality column: Receiver_account
[DEBUG] x_cat shape: (1030388, 13)


In [19]:
train_sampler = build_weighted_sampler(pl.Series(train_df["Is_laundering"]))

In [20]:
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True, collate_fn=collate_encoded_batch)


In [21]:
val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False, collate_fn=collate_encoded_batch)

In [22]:
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False, collate_fn=collate_encoded_batch)

In [23]:
model = TabAMLModel(category_sizes=train_ds.category_sizes, cont_dim=len(cont_cols), embedding_dim=64, shared_ratio=1/6, num_heads=4, num_layers1=2, num_layers2=2, dropout=0.25, micro_indices=(0, 1))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

TabAMLModel(
  (embeddings): ModuleList(
    (0-1): 2 x Embedding(50000, 54)
    (2-3): 2 x Embedding(14, 54)
    (4-5): 2 x Embedding(19, 54)
    (6): Embedding(8, 54)
    (7): Embedding(32, 54)
    (8): Embedding(10, 54)
    (9): Embedding(3, 54)
    (10): Embedding(25, 54)
    (11): Embedding(8, 54)
    (12): Embedding(3, 54)
  )
  (micro_encoder): ResidualAttentionEncoder(
    (layers): ModuleList(
      (0-1): 2 x ResidualAttentionLayer(
        (qkv): Linear(in_features=64, out_features=192, bias=False)
        (attn_out): Linear(in_features=64, out_features=64, bias=False)
        (ffn): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.25, inplace=False)
          (3): Linear(in_features=256, out_features=64, bias=True)
          (4): Dropout(p=0.25, inplace=False)
        )
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e

In [24]:
pos_weight = compute_pos_weight(pl.Series(train_df["Is_laundering"]))
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], device=device))

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=5e-6)

In [ ]:
epochs = 50
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    val_metrics = evaluate(model, val_loader, loss_fn, device)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_metrics[0]:.4f} | Val ROC-AUC: {val_metrics[1]:.4f}")

Epoch 1/50 | Train Loss: 1.1188 | Val Loss: 1.2114 | Val ROC-AUC: 0.7956
Epoch 2/50 | Train Loss: 0.7435 | Val Loss: 1.4527 | Val ROC-AUC: 0.7999
Epoch 3/50 | Train Loss: 0.5139 | Val Loss: 1.8569 | Val ROC-AUC: 0.8015
Epoch 4/50 | Train Loss: 0.4068 | Val Loss: 1.7017 | Val ROC-AUC: 0.7792


In [ ]:
test_metrics = evaluate(model, test_loader, loss_fn, device)
print("Test Results:", test_metrics)